#  LaTeX-OCR colab reproduction with pytorch lightning

using new trainer file, modified config file and modified dataset file

note:

colab GPU session cut off after ep29, so testing is done in another GPU session

In [1]:
!pwd

/content


In [2]:
%load_ext autoreload
%autoreload 2

# installs

In [3]:
from google.colab import files
import os
from os import path as osp

In [4]:
# for laziness
!pip install pix2tex[train] -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.9/172.9 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.7 MB/s eta 0:00:00


In [5]:
!pip install --upgrade --no-cache-dir gdown -q
!pip install timm==0.5.4 -U -q
!pip install opencv-python-headless -U -q
!pip install lightning==2.0.8 -q
!pip install wandb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.0/660.0 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.7/727.7 kB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# clone repo

In [6]:
%cd /content
!git clone https://github.com/lukas-blecher/LaTeX-OCR

/content
Cloning into 'LaTeX-OCR'...
remote: Enumerating objects: 1834, done.
remote: Counting objects: 100% (806/806), done.
remote: Compressing objects: 100% (246/246), done.
remote: Total 1834 (delta 659), reused 560 (delta 560), pack-reused 1028
Receiving objects: 100% (1834/1834), 9.40 MiB | 20.02 MiB/s, done.
Resolving deltas: 100% (1045/1045), done.


# form dataset

brief notes on dataset format

- text file contains space-separated latex symbols, each line corresponds to a figure

- image files must be named according to line number, line 1 -> 00000.png, line 2 -> 00001.png, etc.

- image files can correspond to a subset of math lines, as below

In [7]:
%%time
if not osp.exists("/content/pdf"):
    !mkdir /content/pdf
    %cd /content/pdf
    !gdown -O pdf.zip 176PKaCUDWmTJdQwc-OfkO0y8t4gLsIvQ
    !gdown -O math.txt 1QUjX6PFWPa-HBWdcY-7bA5TRVUnbyS1D
    !unzip -q pdf.zip
    %cd /content

/content/pdf
Downloading...
From (uriginal): https://drive.google.com/uc?id=176PKaCUDWmTJdQwc-OfkO0y8t4gLsIvQ
From (redirected): https://drive.google.com/uc?id=176PKaCUDWmTJdQwc-OfkO0y8t4gLsIvQ&confirm=t&uuid=0049a857-d51a-46e9-a9a4-9da8d5f9cc09
To: /content/pdf/pdf.zip
100% 284M/284M [00:02<00:00, 97.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QUjX6PFWPa-HBWdcY-7bA5TRVUnbyS1D
To: /content/pdf/math.txt
100% 36.6M/36.6M [00:00<00:00, 65.6MB/s]
/content
CPU times: user 176 ms, sys: 29.7 ms, total: 205 ms
Wall time: 22.9 s


In [8]:
%cd /content/LaTeX-OCR
!python -m pix2tex.dataset.dataset -i /content/pdf/train -e /content/pdf/math.txt \
    -o /content/pdf/train.pkl

/content/LaTeX-OCR
Generate dataset
100% 158480/158480 [00:03<00:00, 49217.76it/s]


In [9]:
%cd /content/LaTeX-OCR
!python -m pix2tex.dataset.dataset -i /content/pdf/val -e /content/pdf/math.txt \
    -o /content/pdf/val.pkl

/content/LaTeX-OCR
Generate dataset
100% 6765/6765 [00:00<00:00, 27958.67it/s]


In [10]:
%cd /content/LaTeX-OCR
!python -m pix2tex.dataset.dataset -i /content/pdf/test -e /content/pdf/math.txt \
    -o /content/pdf/test.pkl

/content/LaTeX-OCR
Generate dataset
100% 30637/30637 [00:00<00:00, 34865.56it/s]


# load whole log directory from drive (for continue training only)

In [11]:
from google.colab import drive
drive.mount('/content/drive')

import shutil
from os import path as osp

drive_dir = "/content/drive/MyDrive/latex_ocr" # the partent dir in drive, change it
log_dir = "reproducing/version_1" # in my own storage, the version is "version_1"

src = osp.join(drive_dir, log_dir)
dst = osp.join("/content/lightning_logs", log_dir)
shutil.copytree(src, dst)

drive.flush_and_unmount()

Mounted at /content/drive


In [ ]:
1# in case needing to remove it
# import shutil
# shutil.rmtree("/content/lightning_logs")

# run

1. edit hyper parameters and/or I/O locations!

2. move the new trainer and modified dataset file

3. run

pt-lightning:

- would assign a version number if not supplied, the string would be "version_0", etc.

- requires a correct dataset size to trigger validation loop (after "all" training pairs are iterated)



In [ ]:
# open the file in a new tab in colab
from google.colab import files
files.view("/content/colab_config.yaml")

<IPython.core.display.Javascript object>

In [ ]:
# if contine training, change the load_pl_chpt, version number, middlestop in config accordingingly
def xstr(s):
    if s is None:
        return 'None'
    return str(s)

# check configs here
import yaml
with open("/content/colab_config.yaml", "r") as f:
    config_dict = yaml.safe_load(f)
    print(xstr(config_dict["load_pl_chpt"]))
    print(xstr(config_dict["version"]))
    print(xstr(config_dict["middlestop"]))

/content/lightning_logs/reproducing/version_1/checkpoints/epoch=27-step=136975.ckpt
version_1
29


In [ ]:
import shutil
shutil.copy2("/content/colab_train_pl.py", "/content/LaTeX-OCR/pix2tex/train_pl.py")
# copy original dataset script, for reference only
if not osp.exists("/content/dataset_original.py"):
    shutil.copy2("/content/LaTeX-OCR/pix2tex/dataset/dataset.py", "/content/dataset_original.py")
shutil.copy2("/content/pl_dataset.py", "/content/LaTeX-OCR/pix2tex/dataset/dataset.py")
%cd /content/LaTeX-OCR
!python -m pix2tex.train_pl --config /content/colab_config.yaml

# # store to drive, copied from following section
# from google.colab import drive
# drive.mount('/content/drive')

# import shutil
# src = "/content/lightning_logs/reproducing/version_1"
# dst = "/content/drive/MyDrive/latex_ocr/reproducing/version_1"
# shutil.copytree(src, dst, dirs_exist_ok=True)

# drive.flush_and_unmount()

/content/LaTeX-OCR
2023-09-26 14:30:57.990751: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Global seed set to 42
out path = /content/lightning_logs
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory /content/lightning_logs/reproducing/version_1/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
Restoring states from the checkpoint path at /content/lightning_logs/reproducing/version_1/checkpoints/epoch=27-step=136975.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type  | Params
-----------------------------------
0 | pt_model | Model | 25.5 M
-----------------------------------
25.5 M    Tra

# store to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import shutil
src = "/content/lightning_logs/reproducing/version_1"
dst = "/content/drive/MyDrive/latex_ocr/reproducing/version_1"
shutil.copytree(src, dst, dirs_exist_ok=True)

drive.flush_and_unmount()

Mounted at /content/drive


# turn ckpt into pth

In [ ]:
import torch
from tqdm import tqdm
from os import path as osp

def ckpt2pth(cpath, pthpath):
    mdict = torch.load(cpath)["state_dict"]
    nm_dict = dict()
    for k in tqdm(mdict.keys()):
        assert k.find("pt_model.") != -1
        nk = k.split("pt_model.")[1]
        nm_dict[nk] = mdict[k]
    torch.save(nm_dict, pthpath)
    print(f"successfully saved at {osp.realpath(pthpath)}")


In [ ]:
# change them
# cpath = "/content/lightning_logs/reproducing/version_1/checkpoints/ep1.pth"
# pthdesc = "/content/ep1.pth"

# ckpt2pth(cpath=cpath, pthpath=pthdesc)

# evaluate by given script

In [13]:
# times varies from batch to batch and model to model a lot
%cd /content/LaTeX-OCR
!python /content/LaTeX-OCR/pix2tex/eval.py \
    -c /content/lightning_logs/reproducing/version_1/checkpoints/ep29.pth \
    -d /content/pdf/test.pkl \
    -b 16 \
    -n 10000

/content/LaTeX-OCR
BLEU: 0.835, ED: 1.43e-01, ACC: 0.521:  99% 1932/1948 [1:31:58<00:45,  2.86s/it]

[['\\', 'operatorname', '*', '{', 'd', 'e', 't', '}', '\\', 'left', '(', 'M', '_', '{', '1', '2', '}', '\\', 'right', ')', '\\', 'operatorname', '*', '{', 'd', 'e', 't', '}', '\\', 'left', '(', '1', '+', 'M', '_', '{', '2', '}', '\\', 'sigma', 'M', '_', '{', '1', '}', '\\', 'sigma', '\\', 'right', ')', '=', '\\', 'operatorname', '*', '{', 'd', 'e', 't', '}', '\\', 'left', '(', '\\', 'left', '(', 'M', '_', '{', '1', '}', '+', 'M', '_', '{', '2', '}', '\\', 'right', ')', '\\', 'right', ')', '.'], ['e', '^', '{', '-', 'i', 'e', '\\', 'chi', '}', '\\', 'hat', '{', 'H', '}', 'e', '^', '{', 'i', 'e', '\\', 'chi', '}', '-', '\\', 'hat', '{', 'H', '}', '=', 'e', '^', '{', '-', 'i', 'e', '\\', 'chi', '}', '\\', 'tilde', '{', 'H', '}', 'e', '^', '{', 'i', 'e', '\\', 'chi', '}', '-', '\\', 'tilde', '{', 'H', '}', '=', 'e', '^', '{', '-', 'i', 'e', '\\', 'chi', '}', 'H', '_', '{', '0', '}', 'e', '^

# tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/lightning_logs